In [75]:
import pandas as pd
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
import numpy as np

%matplotlib inline

# up figure resolution
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 150

import warnings
warnings.filterwarnings('ignore')

import pyvista as pv
from pyvista import set_plot_theme
set_plot_theme('document')

from pyprojroot import here

In [76]:
ld_backup = pd.read_csv(f"http://data.insideairbnb.com/spain/catalonia/barcelona/2020-02-16/data/listings.csv.gz")
ld = ld_backup.copy()

# convert price dtype
ld['price'] = ld.price.str[1:].replace("$", '')
ld['price'] = ld.price.str.replace(',', '')
ld['price'] = ld.price.astype(float)

# log of price
ld['price_log'] = ld.price.apply(np.log)
ld['price_log'] = ld.price_log.replace([np.inf, -np.inf], np.nan)
ld['price_log'] = ld.price_log.fillna(value=0)


# calculcate deal index
ld['deal_index'] = ld.review_scores_rating / ld.price

# dropna
ld = ld.dropna(subset=['deal_index'])

In [77]:
!export PYVISTA_USE_PANEL=true

## Deal

In [78]:
# https://github.com/pyvista/pyvista-support/issues/16
ld_subset = ld.copy()
coords = ld_subset[['latitude', 'longitude', 'deal_index']].values

poly = pv.PolyData(coords)
poly['order'] = coords[:,2]
poly.points[:,2] = poly.points[:,2]/500

plotter = pv.Plotter(notebook=True)
plotter.add_mesh(poly)
plotter.show()

VTK(vtkCocoaRenderWindow, height=400, sizing_mode='stretch_width')

## Price log

In [79]:
ld_subset = ld.copy()
coords = ld_subset[['latitude', 'longitude', 'price_log']].values

poly = pv.PolyData(coords)
poly['price log'] = coords[:,2]
poly.points[:,2] = poly.points[:,2]/500

plotter = pv.Plotter(notebook=True)
plotter.add_mesh(poly)
plotter.show()

VTK(vtkCocoaRenderWindow, height=400, sizing_mode='stretch_width')

## Gif

In [ ]:
viewup = [0, 0, 1]

p = pv.Plotter()
p.add_mesh(poly)
p.show(auto_close=False)
path = p.generate_orbital_path(factor=2.0, n_points=36, viewup=viewup, shift=0.2)
p.open_gif("../results/orbit.gif")
p.orbit_on_path(path, write_frames=True, viewup=viewup)
p.close()